In [1]:
from stockfish import Stockfish
import numpy as np

In [58]:
strong_stockfish = Stockfish(
    path='/home/tmek1244/InternalProjects/stockfish_15_linux_x64_avx2/stockfish_15_x64_avx2',
    depth=18,
    parameters={
        "Threads": 6,
        "Hash": 1024*4
    }
)
weak_stockfish = Stockfish(
    path='/home/tmek1244/InternalProjects/stockfish_15_linux_x64_avx2/stockfish_15_x64_avx2',
    depth=3,
    parameters={
        "Threads": 1,
        "Hash": 16
    }
)

In [ ]:
import json
import random

starting_position = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"


def convert_move(move):
    from_position = ord(move[0]) - 97 + 8*(int(move[1])-1)
    to_position = ord(move[2]) - 97 + 8*(int(move[3])-1)

    return from_position * 64 + to_position

def convert_evaluation(evaluation):
    return evaluation["Centipawn"] if evaluation["Centipawn"] is not None else 1000*(evaluation["Mate"]//abs(evaluation["Mate"]))

current_moves = [starting_position]

already_done = set()
ITERATION = 100
MOVES_PER_POSITION = 10
MOVES_IN_OUTPUT = 5

for i in range(ITERATION):
    all_positions = set(current_moves)
    current_moves = []
    data = []
    print(f"MOVE {i}, total postions: {len(all_positions)} ")
    for i, position in enumerate(all_positions):
        weak_stockfish.set_fen_position(position)
        pool = weak_stockfish.get_top_moves(MOVES_PER_POSITION)
        print(i, end=' ')
        for move in pool:
            strong_stockfish.set_fen_position(position)
            strong_stockfish.make_moves_from_current_position([move['Move']])
            data_position = strong_stockfish.get_fen_position()

            if data_position in already_done:
                continue
            
            already_done.add(data_position)
            evaluation = strong_stockfish.get_evaluation()
            data_value = evaluation["value"] if evaluation["type"] == "cp" else 1000*(evaluation["value"]//abs(evaluation["value"]))
            top_moves = strong_stockfish.get_top_moves(MOVES_IN_OUTPUT)
            top_moves_converted = [(convert_move(top_move['Move']), convert_evaluation(top_move)) for top_move in top_moves]

            current_moves.append(data_position)
            data.append([data_position, data_value, top_moves_converted])
    print("\nCopying...")
    with open('../../chess_data/fen_eval_top_5_100x10_unique.json', 'a') as f:
        for move in data:
            f.write(str(move)+'\n')
        
    current_moves = random.sample(current_moves, min(100, len(current_moves)))